# Installation

In [1]:
!pip install cirq

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 57 kB 1.1 MB/s 
     |████████████████████████████████| 66 kB 810 kB/s 
     |████████████████████████████████| 594 kB 39.7 MB/s 
     |████████████████████████████████| 1.8 MB 38.9 MB/s 
     |████████████████████████████████| 576 kB 44.2 MB/s 
     |████████████████████████████████| 120 kB 58.2 MB/s 
     |████████████████████████████████| 221 kB 51.9 MB/s 
     |████████████████████████████████| 147 kB 59.2 MB/s 
     |████████████████████████████████| 44 kB 3.1 MB/s 
     |████████████████████████████████| 229 kB 53.3 MB/s 
     |████████████████████████████████| 49 kB 5.8 MB/s 
     |████████████████████████████████| 84 kB 3.8 MB/s 
     |████████████████████████████████| 68 kB 7.3 MB/s 
     |████████████████████████████████| 80 kB 8.9 MB/s 
     |████████████████████████████████| 58 kB 5.9 MB/s 
     |████████████████████████████████| 119 kB 

# Import Packages

In [2]:
import cirq
import numpy as np

print('cirq version',cirq.__version__)
print('numpy version',np.__version__)

cirq version 1.0.0
numpy version 1.21.6


# Quantum Fourier Transform

In [3]:
## This code is cloned from the work done in https://github.com/Apress/quantum-machine-learning-python/blob/main/Chapter_4/listing4_1/quantum_fourier_transform.py 
class QFT:
    """
    Quantum Fourier Transform
    Builds the QFT circuit iteratively 
    """

    def __init__(self, signal_length=16,
                 basis_to_transform='',
                 validate_inverse_fourier=False,
                 qubits=None):
        
        self.signal_length = signal_length
        self.basis_to_transform = basis_to_transform
        
        if qubits is None:
            self.num_qubits = int(np.log2(signal_length))
            self.qubits = [cirq.LineQubit(i) for i in range(self.num_qubits)]
        else:
            self.qubits = qubits
            self.num_qubits = len(self.qubits)

        self.qubit_index = 0
        self.input_circuit = cirq.Circuit()

        self.validate_inverse_fourier = validate_inverse_fourier
        self.circuit = cirq.Circuit()
        # if self.validate_inverse_fourier:
        self.inv_circuit = cirq.Circuit()

        for k, q_s in enumerate(self.basis_to_transform):
            if int(q_s) == 1:
                # Change the qubit state from 0 to 1 
                self.input_circuit.append(cirq.X(self.qubits[k]))

    def qft_circuit_iter(self):

        if self.qubit_index > 0:
            # Apply the rotations on the prior qubits
            # conditioned on the current qubit
            for j in range(self.qubit_index):
                diff = self.qubit_index - j + 1
                rotation_to_apply = 2.0 / (2.0 ** diff)
                self.circuit.append(cirq.CZ(self.qubits[self.qubit_index],
                                            self.qubits[j]) ** rotation_to_apply)
        # Apply the Hadamard Transform 
        # on current qubit
        self.circuit.append(cirq.H(self.qubits[self.qubit_index]))
        # set up the processing for next qubit
        self.qubit_index += 1

    def qft_circuit(self):

        while self.qubit_index < self.num_qubits:
            self.qft_circuit_iter()
            # See the progression of the Circuit built
            print(f"Circuit after processing Qubit: {self.qubit_index - 1} ")
            print(self.circuit)
        # Swap the qubits to match qft definititon
        self.swap_qubits()
        print("Circuit after qubit state swap:")
        print(self.circuit)
        # Create the inverse Fourier Transform Circuit
        self.inv_circuit = cirq.inverse(self.circuit.copy())

    def swap_qubits(self):
        for i in range(self.num_qubits // 2):
            self.circuit.append(cirq.SWAP(self.qubits[i], self.qubits[self.num_qubits - i - 1]))

    def simulate_circuit(self):
        sim = cirq.Simulator()
        result = sim.simulate(self.circuit)
        return result

# Quantum Phase Estimation

In [4]:
class QPE:
    
    def __init__(self, num_input_state_qubits=3,
                 num_ancillia_qubits=5,
                 unitary_transform=None,
                 U=None,
                 input_state=None):

        # Số qubits của thanh ghi đầu tiên như mình đề cập ở trên
        self.num_ancillia_qubits = num_ancillia_qubits
        
        # khởi tạo qubits có giá trị bằng |0> ở thanh ghi thứ nhất
        self.output_qubits = [cirq.LineQubit(i) for i in range(self.num_ancillia_qubits)]
        
        # Khởi tạo cấu trúc mạch cho thuật toán. 
        # Về sau các phép biến đổi sẽ được thêm vào bằng circuit.append()
        self.input_circuit = cirq.Circuit()

        # véc-tơ riêng |\psi> đầu vào
        self.input_state =  input_state
        
        if self.input_state is not None:
            self.num_input_qubits = len(self.input_state)
        else:
            self.num_input_qubits = num_input_state_qubits
        
        # khởi tạo qubits để mã hóa véc-tơ |\psi> đầu vào
        self.input_qubits = [cirq.LineQubit(i) for i in
                             range(self.num_ancillia_qubits,
                                   self.num_ancillia_qubits + num_input_state_qubits)]

        # Mã hóa |\psi> theo các giá trị của input_state
        if self.input_state is not None:
            for i, c in enumerate(self.input_state):
                if int(c) == 1:
                    self.input_circuit.append(cirq.X(self.input_qubits[i]))
        
        # Ma trận đơn nhất U đầu vào.
        # Ở đây mọi người có thể tùy ý chọn một ma trận đơn nhất bất kỳ cho tham số U,
        # ngoài ra ở đây mình đưa ra một vài ví dụ cho U: cổng Identity, X, và Z.
        self.unitary_transform = unitary_transform
        if self.unitary_transform is None:
            self.U = cirq.I
        elif self.unitary_transform == 'custom':
            self.U = U
        elif self.unitary_transform == 'Z':
            self.U = cirq.CZ
        elif self.unitary_transform == 'X':
            self.U = cirq.CX
        else:
            raise NotImplementedError(f"self.unitary transform not Implemented")

        self.circuit = cirq.Circuit()
        
    
    def phase_1_create_circuit_iter(self):
        # Triển khai Bước 1 và Bước 2 như mình đã đề cập ở trên
        # Bước 1: Các ancilla qubits bị theo đổi theo cổng Hadamard (cirq.H)
        # Bước 2: Áp dụng phép biến đổi C-U^{2^m} trên input_qubits và bị ràng buộc bới các output_qubits     
        for i in range(self.num_ancillia_qubits):
            self.circuit.append(cirq.H(self.output_qubits[i]))
            _pow_ = 2**(self.num_ancillia_qubits - 1 - i)
            #_pow_ = 2 ** (i)
            for k in range(self.num_input_qubits):
                print(self.U)
                self.circuit.append(self.U(self.output_qubits[i], self.input_qubits[k])**_pow_)
        
        
    def inv_qft(self):
      # Phép nghịch đảo của thuật toán QFT.
      # Mình sẽ lấy code của bài trước, mọi người có thể xem qua ở https://github.com/qmlvietnam/CodeforBlog/blob/main/QFT.ipynb
      # để đọc rõ hơn các tính nghịch đảo của QFT. 
        self._qft_ = QFT(qubits=self.output_qubits)
        self._qft_.qft_circuit()


    def simulate_circuit(self,circ):
      # Chạy mô phỏng máy tính lượng tử cho thuật toán
        sim = cirq.Simulator()
        result = sim.simulate(circ)
        return result

# Simulation

In [5]:
num_input_state_qubits=1
num_ancillia_qubits=2
unitary_transform='Z'
U=None
input_state='1'

_QP_ = QPE(num_ancillia_qubits=num_ancillia_qubits,
                                    num_input_state_qubits=num_input_state_qubits,
                                    unitary_transform=unitary_transform,
                                    input_state=input_state)
_QP_.phase_1_create_circuit_iter()

_QP_.inv_qft()

circuit = _QP_.circuit  + _QP_._qft_.inv_circuit
if len(_QP_.input_circuit) > 0:
    circuit = _QP_.input_circuit + circuit

print(circuit)
result = _QP_.simulate_circuit(circuit)
print(result)

CZ
CZ
Circuit after processing Qubit: 0 
0: ───H───
Circuit after processing Qubit: 1 
0: ───H───@───────────
          │
1: ───────@^0.5───H───
Circuit after qubit state swap:
0: ───H───@───────────×───
          │           │
1: ───────@^0.5───H───×───
0: ───────H───@─────────×───────@────────H───
              │         │       │
1: ───────H───┼─────@───×───H───@^-0.5───────
              │     │
2: ───X───────@^0───@────────────────────────
measurements: (no measurements)

qubits: (cirq.LineQubit(0), cirq.LineQubit(1), cirq.LineQubit(2))
output vector: |101⟩

phase:
output vector: |⟩
